In [1]:
import numpy as np
import matplotlib.pyplot as plt
import keras.utils
import tensorflow as tf

from keras import layers
from keras import models

import anchor as ac
import utils
import preprocessing as pp

import importlib
importlib.reload(ac)
importlib.reload(pp)
import os

import keras.backend as K

Using TensorFlow backend.


# Model 

In [2]:
#configuration
# for anchors
input_shape = (224, 224, 3) 
ss = [0.1, 0.2, 0.3]
r = [1, 0.5, 1.5, 0.75]
variance = np.array([0.1, 0.1, 0.2, 0.2])
n_boxes = len(r)
mode = 'training'
batch_size = 64


# for data_generator:
mafa_paths = {i: os.path.join('data', i) for i in os.listdir('data')}

# get path to train, test data
train_images_path = mafa_paths['train_images']
train_labels_path = os.path.join(mafa_paths['train_labels'], 'LabelTrainAll.mat')
test_images_path = mafa_paths['test_images']
test_labels_path = os.path.join(mafa_paths['test_labels'], 'LabelTestAll.mat')
train_key = 'label_train'
test_key = 'label_test'

neg_iou_threshold = 0.1
iou_threshold = 0.6



In [3]:
### im using resnet 9 for base model of ssd

def res_block(_input, filters, kernel_size ):
    y = layers.Conv2D(filters= filters, kernel_size= kernel_size, padding= 'same')(_input)
    y = layers.BatchNormalization()(y)
    y = layers.Activation('relu')(y)
    y = layers.Conv2D(filters= filters, kernel_size= kernel_size, padding= 'same')(y)
    y = layers.BatchNormalization()(y)
    _input = layers.Conv2D(filters= filters, kernel_size=1)(_input)
    y = layers.add([_input, y])
    y = layers.Activation('relu')(y)
#     y = layers.SpatialDropout2D(dropout_rate)(y)
    return y

# consider to use spatial dropout
input_data = layers.Input(shape= input_shape)
x = keras.layers.ZeroPadding2D(padding=(3, 3))(input_data)
x = layers.Conv2D(filters= 32, kernel_size= 7, strides= 2, padding= 'valid')(x)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)

x = keras.layers.ZeroPadding2D(padding=(1, 1))(x)
x = layers.MaxPool2D(strides= 2, pool_size= 3)(x)

x1 = res_block(_input= x, filters= 64, kernel_size= 3)
# x1 = layers.MaxPool2D(strides= 2)(x1)
x2 = res_block(_input= x1, filters= 64, kernel_size= 3)
x2 = layers.MaxPool2D(strides= 2)(x2)

x3 = res_block(_input= x2, filters= 128, kernel_size= 3)
x4 = res_block(_input= x3, filters= 128, kernel_size= 3)

x5 = layers.Conv2D(filters= 64, kernel_size= 3, padding= 'same')(x4)
x5 = layers.BatchNormalization()(x5)
x5 = layers.Activation('relu')(x5)
x5_1 = layers.MaxPool2D(strides= 2)(x5)

x6 = layers.Conv2D(filters= 64, kernel_size= 3, padding= 'same')(x5_1)
x6 = layers.BatchNormalization()(x6)
x6 = layers.Activation('relu')(x6)
x6_1 = layers.MaxPool2D(strides= 2)(x6)

x7 = layers.Conv2D(filters= 64, kernel_size= 3, padding= 'same')(x6_1)
x7 = layers.BatchNormalization()(x7)
x7 = layers.Activation('relu')(x7)

# output = layers.Reshape((-1, 2))(x7)
# print(output)

## bow build upon some predictor layers (b, total_boxes, 2 + 4)
pretictor_layers = [x5, x6, x7]

cls_preds = [layers.Conv2D(n_boxes*2, 3, padding= 'same')(x) for x in pretictor_layers ]
# reshape each predict to(b, -1, 2)
cls_preds = [layers.Reshape((-1, 2))(pred) for pred in cls_preds]
# concat all cls pred (b, total_box, 2)
cls_pred = layers.Concatenate(axis= 1)(cls_preds)

reg_preds = [layers.Conv2D(n_boxes*4, 3, padding= 'same')(x) for x in pretictor_layers ]
# reshape each predict to(b, -1, 2)
reg_preds = [layers.Reshape((-1, 4))(pred) for pred in reg_preds]
# concat all cls pred (b, total_box, 2)
reg_pred = layers.Concatenate(axis= 1)(reg_preds)


if mode == 'training':
    #concat cls and boxes coordinates prediction
    output = layers.Concatenate(axis= -1)([cls_pred, reg_pred])
    
elif mode == 'predicting':
    
    ### anchor boxes for inference (b, total_boxes, 8)
    anchor_tensors = [ac.gen_anchor_tensor(x, s, r, variance) for x, s in zip(pretictor_layers,ss)]
    anchor_tensor = layers.Concatenate(axis= 1)(anchor_tensors)
    
    # concat anchor tensor to prediction for inference purpose
    output = layers.Concatenate(axis= -1)([cls_pred, reg_pred, anchor_tensor])
else: 
    raise ValueError('not a valid mode')
    
model = models.Model(inputs = input_data, outputs = output)

# loss and compile

In [4]:
importlib.reload(utils)
loss = utils.LossSSD()
model.compile(optimizer=keras.optimizers.RMSprop(0.01), loss=loss.compute_loss)

# Data Generator

In [50]:
### get fmap_shapes and anchor_boxes for labeling
importlib.reload(ac)
fmap_shapes = [(x._keras_shape[1], x._keras_shape[2]) for x in pretictor_layers]
anchor_boxes = ac.all_anchor_boxes(fmap_shapes, ss, r)
label_anchor = ac.LabelAnchor(anchor_boxes, variance, iou_threshold, neg_iou_threshold)

In [56]:
importlib.reload(pp)
train_generator = pp.DataGenerator(images_path = train_images_path, 
                                   labels_path = train_labels_path, 
                                   key = 'label_train',
                                   label_anchor= label_anchor)

In [57]:
importlib.reload(pp)
test_generator = pp.DataGenerator(images_path = test_images_path, 
                                   labels_path = test_labels_path, 
                                   key = 'LabelTest',
                                   label_anchor = label_anchor)

# set up and run

In [58]:

model.fit_generator(generator= train_generator, 
                    validation_data= test_generator)


NotImplementedError: 

In [59]:
import pdb
pdb.pm()

> /usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py(368)__len__()
-> raise NotImplementedError
(Pdb) l
363  	        """Number of batch in the Sequence.
364  	
365  	        # Returns
366  	            The number of batches in the Sequence.
367  	        """
368  ->	        raise NotImplementedError
369  	
370  	    def on_epoch_end(self):
371  	        """Method called at the end of every epoch.
372  	        """
373  	        pass
--KeyboardInterrupt--
(Pdb) c
